In [15]:
from dotenv import load_dotenv
load_dotenv()
import logging
from bs4 import BeautifulSoup
import html2text
import httpx
import yaml
import json
from pydantic import Field, BaseModel
from langgraph.graph import MessagesState, StateGraph, START, END
from typing import List
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.types import Send
import operator
from typing import Annotated
from typing import NamedTuple
from utils.fetch_docs import fetch_documents
from langchain_google_genai import ChatGoogleGenerativeAI





# Set up the logger
logging.basicConfig(
    level=logging.INFO,  # Set to DEBUG for detailed logs
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        # logging.FileHandler("scraper.log"),  # Log to a file
        logging.StreamHandler()  # Log to console
    ]
)

logger = logging.getLogger(__name__)


template = """Your job is to get information from a user about what kind of agent they wish to build.

You should get the following information from them:

- What the objective of the agent is
- Various usecases of the agent 
- Some examples of what the agent will be doing (Input and expected output pairs)

If you are not able to discern this info, ask them to clarify! Do not attempt to wildly guess.

After you are able to discern all the information, call the tool AgentInstruction"""

In [73]:
class AgentInstructions(BaseModel):
    """Instructions on how to build the Agent"""
    objective: str = Field(description= "What is the primary objective of the agent")
    usecases: List[str] = Field(description= "What are the various responsibilities of the agent which it needs to fulfill")
    examples : str = Field(description= "What are some examples of the usage of the agent (input query and expected output from the agent) ?")

class AgentBuilderState(MessagesState):
    agent_instructions: AgentInstructions = Field("the requirement analysis generated by the model.")
    json_code: str = Field("The json code generated")
    python_code: str = Field("The Python code generated")

class ArchitectureEvaluationState(MessagesState):
    agent_instructions: AgentInstructions = Field("the requirement analysis generated by the model.")
    url: str = Field("url of the agent architecture to evaluate against")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20", temperature=0)

def requirement_analysis_node(state: AgentBuilderState):
    
    llm_with_tool = llm.bind_tools([AgentInstructions])
    response = llm_with_tool.invoke([SystemMessage(content=template)] + state["messages"])
    
      # Construct the final answer from the arguments of the last tool call  
    if len(response.tool_calls) == 0:
        return {"messages": [response]}
    
    agent_instructions = response.tool_calls[0]
    agent_instructions = AgentInstructions(**agent_instructions["args"])
    
    return {"messages": [response], "agent_instructions": agent_instructions}


In [74]:

def route_state(state: AgentBuilderState):
    messages = state["messages"]
    if isinstance(messages[-1], AIMessage) and messages[-1].tool_calls:
        return "agent_kernel_builder"
    elif not isinstance(messages[-1], HumanMessage):
        return END
    return "requirement_analysis"

In [75]:

AGENT_KERNEL_PROMPT = PromptTemplate.from_template(
    """
        Task Overview:
        Design a langgraph StateGraph object implementing the the best architecture for the given set of requirements, tailored to fulfill the user requirements defined below

        <Requirements>
        Objectives: {objective}
        usecases: {responsibilities}
        examples: {examples}
        </Requirements>

        
        Expected Output:
        Your task is to produce a compiled StateGraph object.

        Guidelines for Code Generation:
        - Accuracy: Avoid hallucinations or speculative assumptions when writing code. Refer exclusively to the provided documentation.
        - Understanding: Thoroughly comprehend the architecture and examples of code.
        - Customization: Generate code tailored specifically to meet the user requirements.

    """)

In [76]:
def agent_kernel_builder(state: AgentBuilderState):
    """Build the agent kernel using the best architecture."""
    agent_instructions : AgentInstructions = state["agent_instructions"]
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    # agent_architecture_report.name
    #agent_architecture_report.highlights
    #agent_architecture_report.justification
    #agent_architecture_report.tailored_design
    print("reached here")
    response =  llm.invoke([HumanMessage(content=AGENT_KERNEL_PROMPT.format(
        objective=agent_instructions.objective,
        responsibilities=agent_instructions.usecases,
        examples = agent_instructions.examples,
        # langgraph_glossary=fetch_documents(langgraph_glossary_url),
        ))])
    
    # Return the generated agent kernel as the output
    return {
        "messages": [AIMessage(content="Generated agent kernel code!")],
        "python_code": response.content,
    }

In [77]:

CODE_TO_JSON_PROMPT = PromptTemplate.from_template("""
You are tasked with converting the following stategraph comPilation code into a JSON. 

Contextual documents for understanding code:
{documents}

The Input code is as follows:
{code_snippet}

OUTPUT: Explaination and JSON. Do not include any code blocks. Seperate the JSON and explaination blocks and ensure that there is an explaination for each line of JSON produced but keep the blocks seperated.
Each Output JSON will have a nodes sections containing all the nodes and an edges section

Please follow:
1. Produce the explaination first and then the JSON after it. DO not produce the JSON first. 
2. For any conditional edges, please include all the nodes that the source of a conditional edge can reach as part of the explaination.
3. Any Edge entry in the JSON can only be conditional(mention conditional: true) if the source for that edge acts as a source for multiple edges. If you cannot point to atleast 2 targets for 1 source, then that source will not have any conditional edges
4. A source can have any number of targets. Please write the explaination for each source node to target node edge
5. Please ensure that the JSON starts with __START__ node and __END__ node with the correct edges from and to them
6. Ensure all elements in the nodes sections of the output json contain the following fields: Schema_info, input_schema, output_schema, description, function_name. Please do not return any entries in the nodes without these fields and these fields can't be empty
7. Ensure all elements in the edges sections of the output json contain the following fields: source, target, routing_conditions, conditional. Please do not return any entries in the edges without these fields and they can't be empty
8. Every node should be a part of atleast one edge, Please ensure this is followed
9. Attach the code snippet for each node aswell that. Please extract it from the Input code


Example output JSON for a node:
    "code_node":{{
        "schema_info": /"/"/"CodeWriterState:
      type: TypedDict
      fields:
      - name: user_query
        type: str
      - name: execution_result
        type: str/"/"/",
    "input_schema": "CodeWriterState",
    "output_schema":"RequiremenCodeWriterStatetAnalysisState",
    "description":"This node analyzes the user_query, if the query is to write a code, it will make a tool call to run the proposed code. This node returns command object",
    "function_name": "code_step"
    }}

Example output JSON for an edge:
edge:{{ source: "abc", target: "cde", routing_condition: "if abc made a tool call then go to cde", "conditional": true}}
edge:{{ source: "abc", target: "xyz", routing_condition: "if abc made an interupt to a human then go to xyz", "conditional": true}}
edge:{{ source: "xyz", target: "_END_", routing_condition: "no nodes to go after xyz, we have our final output for this path", "conditional": false}}s         
""")

In [78]:
def code_to_json_node(state: AgentBuilderState):
    """Convert the generated code to JSON."""
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    json_code_ouptut = llm.invoke([HumanMessage(content=CODE_TO_JSON_PROMPT.format(
        code_snippet=state["python_code"],
        documents = fetch_documents(langgraph_glossary_url),
        ))])
    
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated JSON code!")],
        "json_code": json_code_ouptut.content,
    }


In [79]:

JSON_CODE_COMBINE_PROMPT = PromptTemplate.from_template("""
You are tasked with verifying that the given edges and nodes mentioned in the JSON follow the input code.

Contextual documents for understanding code:
{documents}

The Input code is as follows:
<Input code>
{code_snippet}
</Input code>
                                                        
<JSON>
{node_json}
</JSON>

OUTPUT: JSON

Please ensure that the JSON follows these rules:
1. Any Edge entry in the JSON can only be conditional(mention conditional: true) if the source for that edge acts as a source for multiple edges. If you cannot point to atleast 2 targets for 1 source, then that source will not have any conditional edges
2. A source can have any number of targets.
3. Please ensure that the JSON starts with __START__ node and __END__ node with the correct edges from and to them
4. Ensure all elements in the nodes sections of the output json contain the following fields: Schema_info, input_schema, output_schema, description, function_name. Please do not return any entries in the nodes without these fields and these fields can't be empty
5. Ensure all elements in the edges sections of the output json contain the following fields: source, target, routing_conditions, conditional. Please do not return any entries in the edges without these fields and they can't be empty
6. Every node should be a part of atleast one edge, Please ensure this is followed
7. Attach the code snippet for each node aswell that. Please extract it from the Input code.
8. Final schema of the json is:
<schema>
Example output JSON for a node:
    "code_node":{{
        "id:: <id of node, similar to name>"
        "schema_info": /"/"/"<input state object>
      type: <overall type>
      fields:
      - name: <name of field>
        type: <type>
      - name: <name of field>
        type: <type>/"/"/",
    "input_schema": <input state object>,
    "output_schema":<output state object>,
    "description":<description>,
    "function_name": <name>
    "code": <python_code>
    }}

Example output JSON for an edge:
edge:{{ source: <source_node>, target: <target_node>, routing_condition: <routing_condition>, "conditional": True/False}}
edge:{{ source: <source_node>, target: <target_node>, routing_condition: <routing_condition>, "conditional": True/False}}
edge:{{ source: <sourse_node">, target: <target_node>, routing_condition: <routing_condition>, "conditional": True/False}}        
</schema>                                               
         
1. You can update the JSON but do not update any pre-exisiting field of JSON unless you have an extremely strong justification for it.
2. If they do not follow the code or if there any missing components then please update the JSON with a justification for it. The justification should draw inspiration from the contextual document mentioned.
3. Please ensure that the conditional edges follow the rules mentioned above                       
4. Also please add a "code" entry in each "code_node" entry of the JSON.
5. "code" entry should have the code of the node that the entry in the JSON is trying to represent
6. Do not update any pre-exisiting field of JSON unless you have an extremely strong justification for it
""")

In [80]:

JSON_CODE_COMBINE_PROMPT = PromptTemplate.from_template("""
You are tasked with verifying and updating the provided JSON that represents the nodes and edges of the langgraph to ensure it is correct with respect to the input code.

Contextual Documents for Understanding Code:
{documents}

Input Code:
<Input code>{code_snippet}</Input code>
JSON:
<JSON>{node_json}</JSON>

OUTPUT: JSON
Your task is divided into two parts:
- Validation: Verify that the JSON adheres to the rules outlined below.
- Correction and Augmentation: If the JSON is incorrect or incomplete, update it with a clear justification for every change, and include the code snippet for each node extracted from the input code.

Rules for Validation and Update:
- Conditional Edges:- An edge can be marked as conditional: true only if its source acts as a source for multiple edges (i.e., at least two targets).
- If the source does not meet this condition, then it cannot have conditional edges.

- Edge Targets:- Each source can have any number of targets. This flexibility must be maintained.

- Start and End Nodes:- The JSON must begin with the __START__ node and conclude with the __END__ node, with correct edges to and from them. Edges into the END node can also be conditional if they meet the above mentioned conditions      

- Node Structure:- Each node entry in the nodes section of the JSON must include the following non-empty fields:- schema_info
- id
- schema_info
- input_schema
- output_schema
- description
- function_name


- Edge Structure:- Each edge entry in the edges section of the JSON must include the following non-empty fields:- source
- target
- routing_conditions
- conditional


- Node-Edge Relationship:- Every node must be part of at least one edge. Ensure this relationship is consistently followed.

- Node Code Snippets:- Attach a code field to every node in the JSON, extracted directly from the input code.

- Schema Requirements:- The final JSON must conform to the schema provided below:


Schema for JSON. Ensure the following schema is followed. No field should be missing for any node or edge:
- Node Example:

{{
  "code_node": {{
    "id" : "<id of the node, similar to name>"
    "schema_info": "<define the class structure of the state of this node. Also provide a name>",
    "input_schema": "<input state object name>",
    "output_schema": "<output state object name>",
    "description": "<description>",
    "function_name": "<function_name>",
    "code": "<python_code>"
  }}
}}


- Edge Example:

{{
  "edge": {{
    "source": "<source_node>",
    "target": "<target_node>",
    "routing_condition": "<routing_condition>",
    "conditional": true/false
  }}
}}


                                            
Key Instructions:
- Do not update any pre-existing field of the JSON unless you have an extremely strong justification for doing so.
- Clearly document the reasoning behind any additions, updates, or modifications to the JSON. Justifications should draw inspiration from the contextual documents mentioned earlier.
- Ensure conditional edges strictly adhere to the rules outlined above.
- Input_Schema and output_schemas can only have value None in JSON for START and END nodes. Please follow this without fail
- Include a code field for each code_node entry, with the exact code that corresponds to the node in the input code.


""")



In [81]:
def json_better_node(state: AgentBuilderState):
    """Add code to the json flow"""
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    json_code_ouptut = llm.invoke([HumanMessage(content=JSON_CODE_COMBINE_PROMPT.format(
        code_snippet=state["python_code"],
        documents = fetch_documents(langgraph_glossary_url),
        node_json = state["json_code"]
        ))])
    
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated updated JSON code!")],
        "json_code": json_code_ouptut.content,
    }


In [82]:
workflow = StateGraph(AgentBuilderState)
workflow.add_node("requirement_analysis", requirement_analysis_node)
workflow.add_node("agent_kernel_builder", agent_kernel_builder)
workflow.add_node("code_to_json", code_to_json_node)
workflow.add_node("json_update", json_better_node)

@workflow.add_node
def add_tool_message(state: AgentBuilderState):
    
   
    return {
        "messages": [
            ToolMessage(
                content="Requirements generated!",
                tool_call_id=state["messages"][-1].tool_calls[0]["id"],
            )
        ]
    }

workflow.add_edge("json_update", END)
workflow.add_edge("code_to_json", "json_update")
workflow.add_edge("agent_kernel_builder", "code_to_json")
workflow.add_conditional_edges("requirement_analysis", route_state, ["agent_kernel_builder", "requirement_analysis", END])
workflow.add_edge(START, "requirement_analysis")
infograph = workflow.compile()

In [83]:
import uuid

cached_human_responses = ["hi!", "rag prompt", "1 rag, 2 none, 3 no, 4 no", "red", "q"]
cached_response_index = 0
config = {"configurable": {"thread_id": str(uuid.uuid4())}}
while True:
    try:
        user = input("User (q/Q to quit): ")
    except:
        user = cached_human_responses[cached_response_index]
        cached_response_index += 1
    print(f"User (q/Q to quit): {user}")
    if user in {"q", "Q"}:
        print("AI: Byebye")
        break
    output = None
    for output in infograph.stream(
        {"messages": [HumanMessage(content=user)]}, config=config, stream_mode="updates"
    ):
        last_message = next(iter(output.values()))["messages"][-1]
        last_message.pretty_print()

    if output and "prompt" in output:
        print("Done!")
        

User (q/Q to quit): 1. Primary objective is handle customers, resolve their issues, fetch information for them and conflict to resolution 2. Use cases would be updating customer information, providing customers with the data they ask for, asking customers for their requirements and suggesting them products 3. example 1: input: what is the price of product x? output:  price of x is 300$ example 2: input: Please update my address to "delhi" output: Address updated, here are your new user details-> Name: abc, age 24, Address: Delhi Example 3: I am opening up a bakery and need to setup my supply chain and billing output: We have product x that will output your supply chain monitoring and product Y that would automate your billing and accoutning along with, I would also suggest product z which helps with customer out reach and is generally used by customers opening new business and fits your use case
================================== Ai Message ==================================
Tool Calls

2025-05-25 18:35:24,644 - INFO - HTTP Request: GET https://langchain-ai.github.io/langgraph/concepts/low_level/ "HTTP/1.1 200 OK"


================================== Ai Message ==================================

Generated JSON code!


2025-05-25 18:36:01,160 - INFO - HTTP Request: GET https://langchain-ai.github.io/langgraph/concepts/low_level/ "HTTP/1.1 200 OK"


================================== Ai Message ==================================

Generated updated JSON code!
User (q/Q to quit): q
AI: Byebye


In [ ]:
{
  "nodes": {
    "__START__": {
      "schema_info": "None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The entry point of the graph.",
      "function_name": "__START__",
      "code": "__START__"
    },
    "supervisor": {
      "schema_info": "Router: TypedDict with next as a Literal of options",
      "input_schema": "State",
      "output_schema": "Command[Literal[*members, '__end__']]",
      "description": "This node determines which worker to act next based on user input.",
      "function_name": "supervisor_node",
      "code": "def supervisor_node(state: State) -> Command[Literal[*members, '__end__']]:\n    messages = [\n        {\"role\": \"system\", \"content\": system_prompt},\n    ] + state[\"messages\"]\n    response = llm.with_structured_output(Router).invoke(messages)\n    goto = response[\"next\"]\n    if goto == \"FINISH\":\n        goto = END\n    return Command(goto=goto, update={\"next\": goto})"
    },
    "food_logger": {
      "schema_info": "State: MessagesState with next as a string",
      "input_schema": "State",
      "output_schema": "Command[Literal['supervisor']]",
      "description": "This node logs food intake and calculates calories.",
      "function_name": "food_logger_node",
      "code": "def food_logger_node(state: State) -> Command[Literal['supervisor']]:\n    result = food_logger_agent.invoke(state)\n    return Command(\n        update={\n            \"messages\": [\n                HumanMessage(content=result[\"messages\"][-1].content, name=\"food_logger\")\n            ]\n        },\n        goto=\"supervisor\",\n    )"
    },
    "workout_tracker": {
      "schema_info": "State: MessagesState with next as a string",
      "input_schema": "State",
      "output_schema": "Command[Literal['supervisor']]",
      "description": "This node logs workouts and calculates calories burned.",
      "function_name": "workout_tracker_node",
      "code": "def workout_tracker_node(state: State) -> Command[Literal['supervisor']]:\n    result = workout_tracker_agent.invoke(state)\n    return Command(\n        update={\n            \"messages\": [\n                HumanMessage(content=result[\"messages\"][-1].content, name=\"workout_tracker\")\n            ]\n        },\n        goto=\"supervisor\",\n    )"
    },
    "__END__": {
      "schema_info": "None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The termination point of the graph.",
      "function_name": "__END__",
      "code": "__END__"
    }
  },
  "edges": [
    {
      "source": "__START__",
      "target": "supervisor",
      "routing_conditions": "Start the graph execution by invoking the supervisor node.",
      "conditional": false
    },
    {
      "source": "supervisor",
      "target": "food_logger",
      "routing_conditions": "If the supervisor decides to log food, go to food_logger.",
      "conditional": true
    },
    {
      "source": "supervisor",
      "target": "workout_tracker",
      "routing_conditions": "If the supervisor decides to log workouts, go to workout_tracker.",
      "conditional": true
    },
    {
      "source": "supervisor",
      "target": "__END__",
      "routing_conditions": "If the supervisor decides to finish, go to __END__.",
      "conditional": false
    },
    {
      "source": "food_logger",
      "target": "supervisor",
      "routing_conditions": "After logging food, return to supervisor for next action.",
      "conditional": false
    },
    {
      "source": "workout_tracker",
      "target": "supervisor",
      "routing_conditions": "After logging workouts, return to supervisor for next action.",
      "conditional": false
    }
  ]
}

In [ ]:
{
  "nodes": {
    "__START__": {
      "schema_info": "Type: None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The starting point of the graph.",
      "function_name": "",
      "code": ""
    },
    "supervisor": {
      "schema_info": "Router: type: TypedDict fields: - name: next type: Literal['food_logger', 'workout_logger', 'FINISH']",
      "input_schema": "MessagesState",
      "output_schema": "Router",
      "description": "This node manages the conversation and routes to the appropriate worker based on user input.",
      "function_name": "supervisor_node",
      "code": "def supervisor_node(state: State) -> Command[Literal[*members, '__end__']]:\n    messages = [\n        {'role': 'system', 'content': system_prompt},\n    ] + state['messages']\n    response = llm.with_structured_output(Router).invoke(messages)\n    goto = response['next']\n    if goto == 'FINISH':\n        goto = END\n\n    return Command(goto=goto, update={'next': goto})"
    },
    "food_logger": {
      "schema_info": "Type: MessagesState",
      "input_schema": "MessagesState",
      "output_schema": "MessagesState",
      "description": "This node logs food items and returns the result to the supervisor.",
      "function_name": "food_logging_node",
      "code": "def food_logging_node(state: State) -> Command[Literal['supervisor']]:\n    food_item = state['messages'][-1]['content']  # Get the last user input\n    result = food_logging_agent_tool.invoke(food_item)\n    return Command(\n        update={\n            'messages': [\n                HumanMessage(content=result, name='food_logger')\n            ]\n        },\n        goto='supervisor',\n    )"
    },
    "workout_logger": {
      "schema_info": "Type: MessagesState",
      "input_schema": "MessagesState",
      "output_schema": "MessagesState",
      "description": "This node logs workout details and returns the result to the supervisor.",
      "function_name": "workout_logging_node",
      "code": "def workout_logging_node(state: State) -> Command[Literal['supervisor']]:\n    workout_details = state['messages'][-1]['content']  # Get the last user input\n    result = workout_analysis_tool.invoke(workout_details)\n    return Command(\n        update={\n            'messages': [\n                HumanMessage(content=result, name='workout_logger')\n            ]\n        },\n        goto='supervisor',\n    )"
    },
    "__END__": {
      "schema_info": "Type: None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The end point of the graph.",
      "function_name": "",
      "code": ""
    }
  },
  "edges": [
    { "source": "__START__", "target": "supervisor", "routing_condition": "Start the conversation by invoking the supervisor.", "conditional": false },
    { "source": "supervisor", "target": "food_logger", "routing_condition": "If the supervisor routes to food_logger based on user input.", "conditional": true },
    { "source": "supervisor", "target": "workout_logger", "routing_condition": "If the supervisor routes to workout_logger based on user input.", "conditional": true },
    { "source": "supervisor", "target": "__END__", "routing_condition": "If the supervisor routes to FINISH, end the conversation.", "conditional": false },
    { "source": "food_logger", "target": "supervisor", "routing_condition": "After logging food, return to the supervisor.", "conditional": false },
    { "source": "workout_logger", "target": "supervisor", "routing_condition": "After logging workout, return to the supervisor.", "conditional": false }
  ]
}

In [ ]:
```json
{
  "nodes": {
    "__START__": {
      "id": "__START__",
      "schema_info": null,
      "input_schema": null,
      "output_schema": null,
      "description": "Represents the entry point of the graph. This is a virtual node.",
      "function_name": "START",
      "code": null
    },
    "__END__": {
      "id": "__END__",
      "schema_info": null,
      "input_schema": null,
      "output_schema": null,
      "description": "Represents the termination point of the graph. This is a virtual node.",
      "function_name": "END",
      "code": null
    },
    "intent_router": {
      "id": "intent_router",
      "schema_info": "State:\n  type: TypedDict\n  fields:\n  - name: user_input\n    type: str\n  - name: parsed_intent\n    type: str # 'log_diet', 'log_workout', 'analyze_workout', 'unclear'\n  - name: diet_log\n    type: List[FoodItem]\n  - name: workout_log\n    type: List[WorkoutExercise]\n  - name: daily_calories_consumed\n    type: int\n  - name: daily_calories_burnt\n    type: int\n  - name: weekly_workout_summary\n    type: Dict[str, Any] # e.g., {'bicep_curls_total_sets': 10, 'bicep_goal_achieved': False}\n  - name: analysis_result\n    type: str\n  - name: response\n    type: str\nFoodItem:\n  type: TypedDict\n  fields:\n  - name: name\n    type: str\n  - name: calories\n    type: int\n  - name: meal_type\n    type: str # e.g., breakfast, lunch, dinner, snack, drink\nWorkoutExercise:\n  type: TypedDict\n  fields:\n  - name: name\n    type: str\n  - name: sets\n    type: int\n  - name: reps\n    type: int\n  - name: calories_burnt\n    type: int\n",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node is responsible for classifying the user's initial input. It uses a mock LLM to determine if the user wants to log diet, log a workout, analyze workouts, or if the intent is unclear.",
      "function_name": "intent_router",
      "code": "def intent_router(state: State) -> State:\n    \"\"\"Determines the user's intent based on the input.\"\"\"\n    prompt = ChatPromptTemplate.from_messages([\n        (\"system\", \"You are an AI assistant that classifies user intent for a diet and workout tracker. Classify the following user input into one of these categories: 'log_diet', 'log_workout', 'analyze_workout', 'unclear'. Respond with only the category name.\"),\n        (\"user\", \"{user_input}\")\n    ])\n    chain = prompt | mock_llm.invoke | StrOutputParser()\n    intent = chain.invoke({\"user_input\": state[\"user_input\"]})\n    return {\"parsed_intent\": intent}"
    },
    "log_diet": {
      "id": "log_diet",
      "schema_info": "State:\n  type: TypedDict\n  fields:\n  - name: user_input\n    type: str\n  - name: parsed_intent\n    type: str # 'log_diet', 'log_workout', 'analyze_workout', 'unclear'\n  - name: diet_log\n    type: List[FoodItem]\n  - name: workout_log\n    type: List[WorkoutExercise]\n  - name: daily_calories_consumed\n    type: int\n  - name: daily_calories_burnt\n    type: int\n  - name: weekly_workout_summary\n    type: Dict[str, Any] # e.g., {'bicep_curls_total_sets': 10, 'bicep_goal_achieved': False}\n  - name: analysis_result\n    type: str\n  - name: response\n    type: str\nFoodItem:\n  type: TypedDict\n  fields:\n  - name: name\n    type: str\n  - name: calories\n    type: int\n  - name: meal_type\n    type: str # e.g., breakfast, lunch, dinner, snack, drink\nWorkoutExercise:\n  type: TypedDict\n  fields:\n  - name: name\n    type: str\n  - name: sets\n    type: int\n  - name: reps\n    type: int\n  - name: calories_burnt\n    type: int\n",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node processes user input related to diet. It extracts food items, estimates their calorie content, and updates the `diet_log` and `daily_calories_consumed` fields in the `State`. It also constructs a user-friendly response message.",
      "function_name": "log_diet",
      "code": "def log_diet(state: State) -> State:\n    \"\"\"Processes diet-related input, updates diet_log and daily_calories_consumed.\"\"\"\n    prompt = ChatPromptTemplate.from_messages([\n        (\"system\", \"Extract food items and estimated calories from the user input. Provide a JSON array of objects with 'name', 'calories' (integer), and 'meal_type' (e.g., 'breakfast', 'lunch', 'dinner', 'snack', 'unknown'). Example: [{\\\"name\\\": \\\"apple\\\", \\\"calories\\\": 95, \\\"meal_type\\\": \\\"snack\\\"}]. If no specific meal type is mentioned, use 'unknown'. If multiple items, list them all. If no food is mentioned, return empty array.\"),\n        (\"user\", \"{user_input}\")\n    ])\n    chain = prompt | mock_llm.invoke | StrOutputParser()\n    \n    try:\n        extracted_data_str = chain.invoke({\"user_input\": state[\"user_input\"]})\n        extracted_food_items = json.loads(extracted_data_str)\n    except json.JSONDecodeError:\n        extracted_food_items = []\n\n    current_diet_log = state.get(\"diet_log\", [])\n    current_calories = state.get(\"daily_calories_consumed\", 0)\n\n    logged_items_names = []\n    for item in extracted_food_items:\n        current_diet_log.append(item)\n        current_calories += item.get(\"calories\", 0)\n        logged_items_names.append(item.get(\"name\", \"unknown item\"))\n\n    if logged_items_names:\n        response_msg = f\"Logged {', '.join(logged_items_names)} as part of your diet.\"\n    else:\n        response_msg = \"Could not identify any food items to log.\"\n    \n    response_msg += f\" Calories consumed today: {current_calories}.\"\n\n    return {\n        \"diet_log\": current_diet_log,\n        \"daily_calories_consumed\": current_calories,\n        \"response\": response_msg\n    }"
    },
    "log_workout": {
      "id": "log_workout",
      "schema_info": "State:\n  type: TypedDict\n  fields:\n  - name: user_input\n    type: str\n  - name: parsed_intent\n    type: str # 'log_diet', 'log_workout', 'analyze_workout', 'unclear'\n  - name: diet_log\n    type: List[FoodItem]\n  - name: workout_log\n    type: List[WorkoutExercise]\n  - name: daily_calories_consumed\n    type: int\n  - name: daily_calories_burnt\n    type: int\n  - name: weekly_workout_summary\n    type: Dict[str, Any] # e.g., {'bicep_curls_total_sets': 10, 'bicep_goal_achieved': False}\n  - name: analysis_result\n    type: str\n  - name: response\n    type: str\nFoodItem:\n  type: TypedDict\n  fields:\n  - name: name\n    type: str\n  - name: calories\n    type: int\n  - name: meal_type\n    type: str # e.g., breakfast, lunch, dinner, snack, drink\nWorkoutExercise:\n  type: TypedDict\n  fields:\n  - name: name\n    type: str\n  - name: sets\n    type: int\n  - name: reps\n    type: int\n  - name: calories_burnt\n    type: int\n",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node processes user input related to workouts. It extracts exercise details (sets, reps, estimated calories burnt), updates the `workout_log`, `daily_calories_burnt`, and `weekly_workout_summary` fields in the `State`. It also checks for and reports on workout goals and generates a response message.",
      "function_name": "log_workout",
      "code": "def log_workout(state: State) -> State:\n    \"\"\"Processes workout-related input, updates workout_log and daily_calories_burnt.\"\"\"\n    prompt = ChatPromptTemplate.from_messages([\n        (\"system\", \"Extract workout exercises, sets, reps, and estimated calories burnt from the user input. Provide a JSON array of objects with 'name', 'sets' (integer), 'reps' (integer), and 'calories_burnt' (integer). Example: [{\\\"name\\\": \\\"bicep curls\\\", \\\"sets\\\": 4, \\\"reps\\\": 10, \\\"calories_burnt\\\": 80}]. If no specific reps are mentioned, use 10. If no calories are mentioned, estimate based on typical exercise intensity (e.g., 20-50 calories per set).\"),\n        (\"user\", \"{user_input}\")\n    ])\n    chain = prompt | mock_llm.invoke | StrOutputParser()\n\n    try:\n        extracted_data_str = chain.invoke({\"user_input\": state[\"user_input\"]})\n        extracted_exercises = json.loads(extracted_data_str)\n    except json.JSONDecodeError:\n        extracted_exercises = []\n\n    current_workout_log = state.get(\"workout_log\", [])\n    current_calories_burnt = state.get(\"daily_calories_burnt\", 0)\n    current_weekly_summary = state.get(\"weekly_workout_summary\", {})\n\n    logged_exercises_names = []\n    for exercise in extracted_exercises:\n        current_workout_log.append(exercise)\n        current_calories_burnt += exercise.get(\"calories_burnt\", 0)\n        logged_exercises_names.append(exercise.get(\"name\", \"unknown exercise\"))\n        \n        # Update weekly summary for analysis (example: bicep curls goal)\n        exercise_name_key = exercise['name'].lower().replace(' ', '_') + '_total_sets'\n        current_weekly_summary[exercise_name_key] = current_weekly_summary.get(exercise_name_key, 0) + exercise['sets']\n        \n    # Simple goal check for bicep training (example: 7 sets per week)\n    if current_weekly_summary.get('bicep_curls_total_sets', 0) >= 7:\n        current_weekly_summary['bicep_goal_achieved'] = True\n    else:\n        current_weekly_summary['bicep_goal_achieved'] = False\n\n    if logged_exercises_names:\n        response_msg = f\"Workout logged, {current_calories_burnt} calories burnt during the workout today.\"\n    else:\n        response_msg = \"Could not identify any workouts to log.\"\n\n    if current_weekly_summary.get('bicep_goal_achieved'):\n        response_msg += \" You have achieved the weekly goal for bicep training.\"\n\n    return {\n        \"workout_log\": current_workout_log,\n        \"daily_calories_burnt\": current_calories_burnt,\n        \"weekly_workout_summary\": current_weekly_summary,\n        \"response\": response_msg\n    }"
    },
    "analyze_workout": {
      "id": "analyze_workout",
      "schema_info": "State:\n  type: TypedDict\n  fields:\n  - name: user_input\n    type: str\n  - name: parsed_intent\n    type: str # 'log_diet', 'log_workout', 'analyze_workout', 'unclear'\n  - name: diet_log\n    type: List[FoodItem]\n  - name: workout_log\n    type: List[WorkoutExercise]\n  - name: daily_calories_consumed\n    type: int\n  - name: daily_calories_burnt\n    type: int\n  - name: weekly_workout_summary\n    type: Dict[str, Any] # e.g., {'bicep_curls_total_sets': 10, 'bicep_goal_achieved': False}\n  - name: analysis_result\n    type: str\n  - name: response\n    type: str\nFoodItem:\n  type: TypedDict\n  fields:\n  - name: name\n    type: str\n  - name: calories\n    type: int\n  - name: meal_type\n    type: str # e.g., breakfast, lunch, dinner, snack, drink\nWorkoutExercise:\n  type: TypedDict\n  fields:\n  - name: name\n    type: str\n  - name: sets\n    type: int\n  - name: reps\n    type: int\n  - name: calories_burnt\n    type: int\n",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node provides insights and recommendations based on the aggregated `weekly_workout_summary`. It generates an analysis output, which also serves as the response message.",
      "function_name": "analyze_workout",
      "code": "def analyze_workout(state: State) -> State:\n    \"\"\"Provides analysis on optimizing workouts.\"\"\"\n    weekly_summary = state.get(\"weekly_workout_summary\", {})\n    analysis_output = \"Workout analysis:\\n\"\n\n    if not weekly_summary or all(v == 0 for k, v in weekly_summary.items() if '_total_sets' in k):\n        analysis_output += \"No workout data available for analysis yet. Log some workouts first!\\n\"\n    else:\n        if weekly_summary.get('bicep_goal_achieved'):\n            analysis_output += \"Great job on your bicep training! You've hit your weekly goal. Consider increasing intensity or trying new variations.\\n\"\n        elif 'bicep_curls_total_sets' in weekly_summary:\n            analysis_output += f\"You've done {weekly_summary['bicep_curls_total_sets']} sets of bicep curls this week. Keep going to hit your goal of 7 sets!\\n\"\n        \n        # Add more sophisticated analysis here based on actual data\n        analysis_output += \"General tip: Ensure you're balancing muscle groups and incorporating rest days for optimal recovery and growth.\\n\"\n\n    return {\n        \"analysis_result\": analysis_output,\n        \"response\": analysis_output # For simplicity, analysis result is the response\n    }"
    },
    "generate_response": {
      "id": "generate_response",
      "schema_info": "State:\n  type: TypedDict\n  fields:\n  - name: user_input\n    type: str\n  - name: parsed_intent\n    type: str # 'log_diet', 'log_workout', 'analyze_workout', 'unclear'\n  - name: diet_log\n    type: List[FoodItem]\n  - name: workout_log\n    type: List[WorkoutExercise]\n  - name: daily_calories_consumed\n    type: int\n  - name: daily_calories_burnt\n    type: int\n  - name: weekly_workout_summary\n    type: Dict[str, Any] # e.g., {'bicep_curls_total_sets': 10, 'bicep_goal_achieved': False}\n  - name: analysis_result\n    type: str\n  - name: response\n    type: str\nFoodItem:\n  type: TypedDict\n  fields:\n  - name: name\n    type: str\n  - name: calories\n    type: int\n  - name: meal_type\n    type: str # e.g., breakfast, lunch, dinner, snack, drink\nWorkoutExercise:\n  type: TypedDict\n  fields:\n  - name: name\n    type: str\n  - name: sets\n    type: int\n  - name: reps\n    type: int\n  - name: calories_burnt\n    type: int\n",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node is responsible for formulating the final user-facing message. It prioritizes any response already set by a preceding action node (like `log_diet` or `log_workout`). If no specific response is available (e.g., for an unclear intent), it provides a default helpful message.",
      "function_name": "generate_response",
      "code": "def generate_response(state: State) -> State:\n    \"\"\"Generates the final user-facing response.\"\"\"\n    # If an action node has already set a specific response, use it.\n    # Otherwise, provide a default message for unclear intents.\n    if state.get(\"response\"):\n        return {\"response\": state[\"response\"]}\n    else:\n        return {\"response\": \"I'm not sure how to help with that. Please tell me if you want to log diet, log a workout, or get a workout analysis.\"}"
    }
  },
  "edges": [
    {
      "source": "__START__",
      "target": "intent_router",
      "routing_conditions": "Always transition to the initial intent classification node.",
      "conditional": false
    },
    {
      "source": "intent_router",
      "target": "log_diet",
      "routing_conditions": "If the 'parsed_intent' is 'log_diet', transition to the 'log_diet' node.",
      "conditional": true
    },
    {
      "source": "intent_router",
      "target": "log_workout",
      "routing_conditions": "If the 'parsed_intent' is 'log_workout', transition to the 'log_workout' node.",
      "conditional": true
    },
    {
      "source": "intent_router",
      "target": "analyze_workout",
      "routing_conditions": "If the 'parsed_intent' is 'analyze_workout', transition to the 'analyze_workout' node.",
      "conditional": true
    },
    {
      "source": "intent_router",
      "target": "generate_response",
      "routing_conditions": "If the 'parsed_intent' is 'unclear', transition directly to the 'generate_response' node.",
      "conditional": true
    },
    {
      "source": "log_diet",
      "target": "generate_response",
      "routing_conditions": "Always transition to the response generation node after logging diet.",
      "conditional": false
    },
    {
      "source": "log_workout",
      "target": "generate_response",
      "routing_conditions": "Always transition to the response generation node after logging workout.",
      "conditional": false
    },
    {
      "source": "analyze_workout",
      "target": "generate_response",
      "routing_conditions": "Always transition to the response generation node after workout analysis.",
      "conditional": false
    },
    {
      "source": "generate_response",
      "target": "__END__",
      "routing_conditions": "The graph terminates after generating the final response.",
      "conditional": false
    }
  ]
}
```

In [ ]:
# {
#   "nodes": {
#     "__START__": {
#       "schema_info": "None",
#       "input_schema": "None",
#       "output_schema": "None",
#       "description": "The starting point of the graph.",
#       "function_name": "start_node"
#     },
#     "supervisor": {
#       "schema_info": "Router: TypedDict with next as a string literal of options.",
#       "input_schema": "State",
#       "output_schema": "State",
#       "description": "This node manages the conversation between workers and decides which worker to route to next.",
#       "function_name": "supervisor_node"
#     },
#     "food_logger": {
#       "schema_info": "State: TypedDict with messages as a list of messages.",
#       "input_schema": "State",
#       "output_schema": "State",
#       "description": "This node logs food intake and calculates calories.",
#       "function_name": "food_logger_node"
#     },
#     "workout_tracker": {
#       "schema_info": "State: TypedDict with messages as a list of messages.",
#       "input_schema": "State",
#       "output_schema": "State",
#       "description": "This node logs workout details and calculates calories burnt.",
#       "function_name": "workout_tracker_node"
#     },
#     "__END__": {
#       "schema_info": "None",
#       "input_schema": "None",
#       "output_schema": "None",
#       "description": "The endpoint of the graph, indicating completion.",
#       "function_name": "end_node"
#     }
#   },
#   "edges": {
#     { 
#       "source": "__START__", 
#       "target": "supervisor", 
#       "routing_conditions": "Start the process by routing to the supervisor node.", 
#       "conditional": false 
#     },
#     { 
#       "source": "supervisor", 
#       "target": "food_logger", 
#       "routing_conditions": "If the supervisor decides to route to food_logger based on user input.", 
#       "conditional": true 
#     },
#     { 
#       "source": "supervisor", 
#       "target": "workout_tracker", 
#       "routing_conditions": "If the supervisor decides to route to workout_tracker based on user input.", 
#       "conditional": true 
#     },
#     { 
#       "source": "supervisor", 
#       "target": "__END__", 
#       "routing_conditions": "If the supervisor decides to finish the process.", 
#       "conditional": true
#     },
#     { 
#       "source": "food_logger", 
#       "target": "supervisor", 
#       "routing_conditions": "After logging food intake, return control to the supervisor.", 
#       "conditional": false 
#     },
#     { 
#       "source": "workout_tracker", 
#       "target": "supervisor", 
#       "routing_conditions": "After logging workout details, return control to the supervisor.", 
#       "conditional": false 
#     }
#   }
# }

In [ ]:
{
  "nodes": {
    "__START__": {
      "schema_info": "",
      "input_schema": "",
      "output_schema": "",
      "description": "Entry point of the graph.",
      "function_name": "__START__",
      "code": ""
    },
    "supervisor": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node manages interactions between the diet and workout teams.",
      "function_name": "teams_supervisor_node",
      "code": "def teams_supervisor_node(state: State) -> Command[Literal[\"diet_logger\"]]: return Command(goto=\"diet_logger\")"
    },
    "diet_logger": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node logs food intake and returns the logged message.",
      "function_name": "diet_logging_node",
      "code": "def diet_logging_node(state: State) -> Command[Literal[\"supervisor\"]]: result = diet_logger_agent.invoke(state) return Command(update={\"messages\": [HumanMessage(content=result[\"messages\"][-1].content, name=\"diet_logger\")]}, goto=\"supervisor\")"
    },
    "workout_logger": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node logs workouts and returns the logged message.",
      "function_name": "workout_logging_node",
      "code": "def workout_logging_node(state: State) -> Command[Literal[\"supervisor\"]]: result = workout_logger_agent.invoke(state) return Command(update={\"messages\": [HumanMessage(content=result[\"messages\"][-1].content, name=\"workout_logger\")]}, goto=\"supervisor\")"
    },
    "diet_supervisor": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node routes to the diet_logger node.",
      "function_name": "diet_supervisor_node",
      "code": "def diet_supervisor_node(state: State) -> Command[Literal[\"diet_logger\"]]: return Command(goto=\"diet_logger\")"
    },
    "workout_supervisor": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node routes to the workout_logger node.",
      "function_name": "workout_supervisor_node",
      "code": "def workout_supervisor_node(state: State) -> Command[Literal[\"workout_logger\"]]: return Command(goto=\"workout_logger\")"
    },
    "diet_team": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node calls the diet team and invokes the diet graph.",
      "function_name": "call_diet_team",
      "code": "def call_diet_team(state: State) -> Command[Literal[\"supervisor\"]]: response = diet_graph.invoke({\"messages\": state[\"messages\"][-1]}) return Command(update={\"messages\": [HumanMessage(content=response[\"messages\"][-1].content, name=\"diet_team\")]}, goto=\"supervisor\")"
    },
    "workout_team": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node calls the workout team and invokes the workout graph.",
      "function_name": "call_workout_team",
      "code": "def call_workout_team(state: State) -> Command[Literal[\"supervisor\"]]: response = workout_graph.invoke({\"messages\": state[\"messages\"][-1]}) return Command(update={\"messages\": [HumanMessage(content=response[\"messages\"][-1].content, name=\"workout_team\")]}, goto=\"supervisor\")"
    },
    "__END__": {
      "schema_info": "",
      "input_schema": "",
      "output_schema": "",
      "description": "Terminal node of the graph.",
      "function_name": "__END__",
      "code": ""
    }
  },
  "edges": [
    { "source": "__START__", "target": "supervisor", "routing_conditions": "Start execution at the supervisor node.", "conditional": false },
    { "source": "supervisor", "target": "diet_supervisor", "routing_conditions": "Route to diet supervisor for diet-related tasks.", "conditional": true },
    { "source": "supervisor", "target": "workout_supervisor", "routing_conditions": "Route to workout supervisor for workout-related tasks.", "conditional": true },
    { "source": "diet_supervisor", "target": "diet_logger", "routing_conditions": "Route to diet logger to log food intake.", "conditional": false },
    { "source": "workout_supervisor", "target": "workout_logger", "routing_conditions": "Route to workout logger to log workouts.", "conditional": false },
    { "source": "diet_team", "target": "supervisor", "routing_conditions": "Return to supervisor after invoking diet graph.", "conditional": false },
    { "source": "workout_team", "target": "supervisor", "routing_conditions": "Return to supervisor after invoking workout graph.", "conditional": false },
    { "source": "diet_logger", "target": "__END__", "routing_conditions": "End process after logging food intake.", "conditional": false },
    { "source": "workout_logger", "target": "__END__", "routing_conditions": "End process after logging workouts.", "conditional": false }
  ]
}

In [ ]:
{
  "nodes": {
    "__START__": {
      "schema_info": "",
      "input_schema": "",
      "output_schema": "",
      "description": "This is the entry point of the graph.",
      "function_name": "start_node"
    },
    "food_logging_node": {
      "schema_info": "MessagesState: type: TypedDict fields: - messages: list[AnyMessage]",
      "input_schema": "MessagesState",
      "output_schema": "MessagesState",
      "description": "This node logs food intake using the food logging agent and updates the messages.",
      "function_name": "food_logging_node"
    },
    "workout_logging_node": {
      "schema_info": "MessagesState: type: TypedDict fields: - messages: list[AnyMessage]",
      "input_schema": "MessagesState",
      "output_schema": "MessagesState",
      "description": "This node tracks workouts using the workout logging agent and updates the messages.",
      "function_name": "workout_logging_node"
    },
    "__END__": {
      "schema_info": "",
      "input_schema": "",
      "output_schema": "",
      "description": "This is the terminal node of the graph.",
      "function_name": "end_node"
    }
  },
  "edges": {
    { "source": "__START__", "target": "food_logging_node", "routing_conditions": "Start the workflow with food logging.", "conditional": false },
    { "source": "food_logging_node", "target": "workout_logging_node", "routing_conditions": "After logging food, transition to workout tracking.", "conditional": true },
    { "source": "workout_logging_node", "target": "__END__", "routing_conditions": "After tracking workouts, conclude the workflow.", "conditional": false }
  }
}

In [ ]:
```json
{
  "nodes": {
    "classify_intent": {
      "id": "classify_intent",
      "schema_info": "GraphState:\n  type: TypedDict\n  fields:\n  - name: input\n    type: str\n  - name: output\n    type: str\n  - name: intent\n    type: Optional[str]\n  - name: customer_data\n    type: dict\n  - name: product_catalog\n    type: dict",
      "input_schema": "GraphState",
      "output_schema": "GraphState",
      "description": "Classifies the intent of the user's input. In a real-world scenario, this function would typically use an LLM (e.g., OpenAI, Anthropic) to perform robust intent classification. For this example, we use simple keyword matching for demonstration.",
      "function_name": "classify_intent",
      "code": "def classify_intent(state: GraphState) -> GraphState:\n    \"\"\"\n    Classifies the intent of the user's input.\n    In a real-world scenario, this function would typically use an LLM\n    (e.g., OpenAI, Anthropic) to perform robust intent classification.\n    For this example, we use simple keyword matching for demonstration.\n    \"\"\"\n    user_input = state[\"input\"].lower()\n    intent = None\n\n    if \"update\" in user_input or \"change my address\" in user_input:\n        intent = \"update_info\"\n    elif \"price of\" in user_input or \"what is\" in user_input or \"data\" in user_input or \"my address\" in user_input or \"my name\" in user_input:\n        intent = \"fetch_data\"\n    elif \"bakery\" in user_input or \"supply chain\" in user_input or \"billing\" in user_input or \"need to setup\" in user_input:\n        intent = \"product_suggestion\"\n    else:\n        intent = \"unclear\" # Fallback for unhandled or ambiguous cases\n\n    print(f\"---CLASSIFY INTENT---\": Input: '{state['input']}' -> Intent: '{intent}'\")\n    return {\"intent\": intent}"
    },
    "update_customer_info_agent": {
      "id": "update_customer_info_agent",
      "schema_info": "GraphState:\n  type: TypedDict\n  fields:\n  - name: input\n    type: str\n  - name: output\n    type: str\n  - name: intent\n    type: Optional[str]\n  - name: customer_data\n    type: dict\n  - name: product_catalog\n    type: dict",
      "input_schema": "GraphState",
      "output_schema": "GraphState",
      "description": "Handles updating customer information based on the parsed input. This function simulates interaction with a customer database.",
      "function_name": "update_customer_info_agent",
      "code": "def update_customer_info_agent(state: GraphState) -> GraphState:\n    \"\"\"\n    Handles updating customer information based on the parsed input.\n    This function simulates interaction with a customer database.\n    \"\"\"\n    user_input = state[\"input\"]\n    customer_data = state[\"customer_data\"].copy() # Create a mutable copy\n    output = \"Could not update information. Please specify what to update.\"\n\n    # Example: \"Please update my address to \"delhi\"\"\n    # Using regex to extract field and value\n    match = re.search(r\"update my (address|email|name|age) to \\\"?([a-zA-Z0-9\\s]+)\\\"\", user_input, re.IGNORECASE)\n    if match:\n        field = match.group(1).lower()\n        value = match.group(2).strip()\n        if field in customer_data:\n            customer_data[field] = value\n            # Construct output matching example format\n            output = (f\"Address updated, here are your new user details-> \"\n                      f\"Name: {customer_data.get('name')}, age {customer_data.get('age')}, \"\n                      f\"Address: {customer_data.get('address')}\")\n        else:\n            output = f\"Sorry, I cannot update the field '{field}'. Available fields are name, age, address, email.\"\n    else:\n        output = \"I understand you want to update information, but I couldn't parse the specific details. Could you please rephrase?\"\n\n    print(f\"---UPDATE CUSTOMER INFO AGENT---\": Output: '{output}'\")\n    return {\"customer_data\": customer_data, \"output\": output}"
    },
    "fetch_info_agent": {
      "id": "fetch_info_agent",
      "schema_info": "GraphState:\n  type: TypedDict\n  fields:\n  - name: input\n    type: str\n  - name: output\n    type: str\n  - name: intent\n    type: Optional[str]\n  - name: customer_data\n    type: dict\n  - name: product_catalog\n    type: dict",
      "input_schema": "GraphState",
      "output_schema": "GraphState",
      "description": "Handles fetching specific information for the customer from mock data.",
      "function_name": "fetch_info_agent",
      "code": "def fetch_info_agent(state: GraphState) -> GraphState:\n    \"\"\"\n    Handles fetching specific information for the customer from mock data.\n    \"\"\"\n    user_input = state[\"input\"].lower()\n    customer_data = state[\"customer_data\"]\n    output = \"I couldn't find the information you asked for.\"\n\n    if \"price of product x\" in user_input:\n        price = customer_data.get(\"product_x_price\")\n        if price:\n            output = f\"price of x is {price}$\"\n    elif \"price of product y\" in user_input:\n        price = customer_data.get(\"product_y_price\")\n        if price:\n            output = f\"price of y is {price}$\"\n    elif \"price of product z\" in user_input:\n        price = customer_data.get(\"product_z_price\")\n        if price:\n            output = f\"price of z is {price}$\"\n    elif \"my address\" in user_input:\n        address = customer_data.get(\"address\")\n        if address:\n            output = f\"Your current address is: {address}\"\n    elif \"my name\" in user_input:\n        name = customer_data.get(\"name\")\n        if name:\n            output = f\"Your name is: {name}\"\n    else:\n        output = \"I can fetch information about product prices or your personal details like address and name. What specific information are you looking for?\"\n\n    print(f\"---FETCH INFO AGENT---\": Output: '{output}'\")\n    return {\"output\": output}"
    },
    "product_suggestion_agent": {
      "id": "product_suggestion_agent",
      "schema_info": "GraphState:\n  type: TypedDict\n  fields:\n  - name: input\n    type: str\n  - name: output\n    type: str\n  - name: intent\n    type: Optional[str]\n  - name: customer_data\n    type: dict\n  - name: product_catalog\n    type: dict",
      "input_schema": "GraphState",
      "output_schema": "GraphState",
      "description": "Asks customers for their requirements and suggests products from the catalog. This function simulates a recommendation engine or a sales agent.",
      "function_name": "product_suggestion_agent",
      "code": "def product_suggestion_agent(state: GraphState) -> GraphState:\n    \"\"\"\n    Asks customers for their requirements and suggests products from the catalog.\n    This function simulates a recommendation engine or a sales agent.\n    \"\"\"\n    user_input = state[\"input\"].lower()\n    product_catalog = state[\"product_catalog\"]\n    output = \"I can help you find products. Please tell me more about your needs.\"\n\n    # Example: \"I am opening up a bakery and need to setup my supply chain and billing\"\n    suggested_products_info = []\n    if \"supply chain\" in user_input:\n        if \"product x\" in product_catalog:\n            suggested_products_info.append(product_catalog[\"product x\"])\n    if \"billing\" in user_input:\n        if \"product y\" in product_catalog:\n            suggested_products_info.append(product_catalog[\"product y\"])\n\n    # Always suggest product z as it fits \"new business\" and \"customer outreach\"\n    if \"product z\" in product_catalog and product_catalog[\"product z\"] not in suggested_products_info:\n        suggested_products_info.append(product_catalog[\"product z\"])\n\n    if suggested_products_info:\n        product_phrases = []\n        for p_info in suggested_products_info:\n            # Format: \"product x that will output your supply chain monitoring\"\n            product_phrases.append(f\"product {p_info['name'].lower().replace('product ', '')} that {p_info['description'].lower()}\")\n\n        # Construct the output string to match the example closely\n        if len(product_phrases) > 1:\n            # Separate the last product for \"along with, I would also suggest\"\n            last_product_phrase = product_phrases.pop()\n            output = \"We have \" + \", \".join(product_phrases)\n            output += f\" along with, I would also suggest {last_product_phrase} and fits your use case\"\n        else:\n            output = f\"We have {product_phrases[0]} and fits your use case\"\n\n        # Add the general \"used by customers opening new business\" part if product z was suggested\n        if any(p['name'] == 'Product Z' for p in suggested_products_info):\n            output = output.replace(\"and fits your use case\", \"and is generally used by customers opening new business and fits your use case\")\n\n    else:\n        output = \"Based on your requirements, I couldn't find specific products. Could you elaborate?\"\n\n    print(f\"---PRODUCT SUGGESTION AGENT---\": Output: '{output}'\")\n    return {\"output\": output}"
    }
  },
  "edges": [
    {
      "source": "__START__",
      "target": "classify_intent",
      "routing_conditions": "This is the entry point of the graph, routing initial user input to the intent classification node.",
      "conditional": false
    },
    {
      "source": "classify_intent",
      "target": "update_customer_info_agent",
      "routing_conditions": "If the classified intent is 'update_info'.",
      "conditional": true
    },
    {
      "source": "classify_intent",
      "target": "fetch_info_agent",
      "routing_conditions": "If the classified intent is 'fetch_data' or if the intent is 'unclear' (fallback).",
      "conditional": true
    },
    {
      "source": "classify_intent",
      "target": "product_suggestion_agent",
      "routing_conditions": "If the classified intent is 'product_suggestion'.",
      "conditional": true
    },
    {
      "source": "update_customer_info_agent",
      "target": "__END__",
      "routing_conditions": "The task of updating customer information is complete.",
      "conditional": false
    },
    {
      "source": "fetch_info_agent",
      "target": "__END__",
      "routing_conditions": "The task of fetching information is complete.",
      "conditional": false
    },
    {
      "source": "product_suggestion_agent",
      "target": "__END__",
      "routing_conditions": "The task of providing product suggestions is complete.",
      "conditional": false
    }
  ]
}
```

In [ ]:
1. Primary objective is handle customers, resolve their issues, fetch information for them and conflict to resolution 2. Use cases would be updating customer information, providing customers with the data they ask for, asking customers for their requirements and suggesting them products 3. example 1: input: what is the price of product x? output:  price of x is 300$ example 2: input: Please update my address to "delhi" output: Address updated, here are your new user details-> Name: abc, age 24, Address: Delhi Example 3: I am opening up a bakery and need to setup my supply chain and billing output: We have product x that will output your supply chain monitoring and product Y that would automate your billing and accoutning along with, I would also suggest product z which helps with customer out reach and is generally used by customers opening new business and fits your use case

In [ ]:
objective: Help users track their diet and workouts     usecases: ['Track diet', 'Store whatever user eats during a day and track calories', 'Track workouts', 'Provide analysis on optimizing workouts']     examples: Example 1: I ate apple today -> Output: Logged 1 apple as part of breakfast, Calories consumed today: 300. Example 2: I did 4 sets of bicep curls and 2 sets of leg presses -> Output: workout logged, 150 calories burnt during the workout today. You have achieved the weekly goal for bicep training.